# ベイズ最適化（hyperopt）でのパラメータ探索

参考資料
- https://qiita.com/nazoking@github/items/f67f92dc60001a43b7dc

## 必要なライブラリのインポート

In [20]:
import numpy as np
import pandas as pd

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

import xgboost as xgb

## データセットの取得

データセットはこちらのURLにアップデートされているものを使用\
https://github.com/ghmagazine/kagglebook/tree/master/input/sample-data

In [5]:
# train_xは学習データ、train_yは目的変数、test_xはテストデータ
train = pd.read_csv('train_preprocessed.csv')
test = pd.read_csv('test_preprocessed.csv')

In [6]:
train.head()

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,1,0,1,0,0,2016,2,13,24194,1
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,1,0,0,2015,7,6,24187,0
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,1,1,1,0,2016,9,17,24201,1


In [7]:
train.shape

(10000, 29)

In [8]:
train_x = train.iloc[:, :28]
train_y = train['target']
test_x = test.copy()

In [11]:
# KFoldクロスバリデーションによる分割の1つを使用し、学習データとバリデーションデータに分ける
kf = KFold(n_splits=4, shuffle=True, random_state=71)
##  トレーニングデータセットとテストデータセットにそれぞれ振り分けた要素番号のリスト
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

## hyperoptライブラリの使用

In [21]:
class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

In [12]:
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}

hyperoptを使用したパラメータ探索は、以下のようになる。
1. チューニングしたいパラメータを引数にとり、最小化したい評価指標のスコアを返す関数を作成する。
  - その関数では、モデルを引数のパラメータで学習させ、バリデーションデータへの予測を行い、評価指標のスコアを計算する処理を行う。
2. hyperoptのfmin関数に、その作成した関数、探索するパラメータの空間、探索回数などを指定することで探索する。

In [17]:
def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする
    
    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])
    
    # Modelクラスを定義しているものとする
    ## Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'pramas: {params}, logloss: {score:.4f}')
    
    # 情報を記録しておく
    history.append((params, score))
    
    return {'loss': score, 'status': STATUS_OK}



In [22]:
# 探索するパラメータの空間を指定する
space2 = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1)
}

# hyperoptによるパラメータ探索の実行
## max_evals: 何回探索するかを指定
max_evals = 10

# Trialsは探索中に計算されたすべての戻り値を検査する。
## trials.trials: 検索のすべてを表す辞書のリスト
## trials.results: 検索中に 'objective'によって返された辞書のリスト
## trials.losses(): 損失の浮動小数点リスト（STATUS_OKとなった場合のscore）
## trials.statuses(): ステータス文字列のリスト
trials = Trials()
history = []
fmin(score, space2, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# (trialsからも情報を取得できるが、パラメータの取得がやや行いづらいため)
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params: {best[0]}, score: {best[1]:.4f}')

[0]	train-error:0.122533	eval-error:0.1424            

[1]	train-error:0.108	eval-error:0.1328               

[2]	train-error:0.101333	eval-error:0.1356            

[3]	train-error:0.0932	eval-error:0.13                

[4]	train-error:0.0884	eval-error:0.1284              

[5]	train-error:0.079867	eval-error:0.126             

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[6]	train-error:0.0744	eval-error:0.1212              

[7]	train-error:0.067467	eval-error:0.1164            

[8]	train-error:0.066133	eval-error:0.1168            

[9]	train-error:0.060533	eval-error:0.112             

pramas: {'gamma': 0.1, 'max_depth': 9, 'min_child_weight': 5.0}, logloss: 0.2800
[0]	train-error:0.154	eval-error:0.1624                                         

[1]	train-error:0.1476	eval-error:0.1604                                        

[2]	train-error:0.1388	eval-error:0.1512                                        

[3]	train-error:0.1416	eval-error:0.1516                                        

[4]	train-error:0.135067	eval-error:0.1456                                      

[5]	train-error:0.130267	eval-error:0.1428                                      

[6]	train-error:0.1272	eval-error:0.1408                                        

[7]	train-error:0.123333	eval-error:0.1416                                      

[8]	train-error:0.116933	eval-error:0.1

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[0]	train-error:0.142133	eval-error:0.1564                                      

[1]	train-error:0.134933	eval-error:0.1524                                      

[2]	train-error:0.125867	eval-error:0.1416                                      

[3]	train-error:0.123067	eval-error:0.142                                       

[4]	train-error:0.116533	eval-error:0.1424                                      

[5]	train-error:0.114133	eval-error:0.142                                       

[6]	train-error:0.110133	eval-error:0.1364                                      

[7]	train-error:0.105333	eval-error:0.1316                                      

[8]	train-error:0.103333	eval-error:0.1288                                      

[9]	train-error:0.098267	eval-error:0.1272                                      

pramas: {'gamma': 0.30000000000000004, 'max_depth': 5, 'min_child_weight': 4.0}, logloss: 0.3147
[0]	train-error:0.131333	eval-error:0.1544                                      

[

...

[9]	train-error:0.073733	eval-error:0.1172                                      

pramas: {'gamma': 0.0, 'max_depth': 6, 'min_child_weight': 1.0}, logloss: 0.3009
[0]	train-error:0.121067	eval-error:0.1408                                      

[1]	train-error:0.111067	eval-error:0.1404                                      

[2]	train-error:0.105733	eval-error:0.1376                                      

[3]	train-error:0.096	eval-error:0.1312                                         

[4]	train-error:0.0892	eval-error:0.1292                                        

[5]	train-error:0.084933	eval-error:0.1248                                      

[6]	train-error:0.080267	eval-error:0.124                                       

[7]	train-error:0.076533	eval-error:0.12                                        

 80%|████████  | 8/10 [00:02<00:00,  3.85trial/s, best loss: 0.2799878002092242]

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[8]	train-error:0.072133	eval-error:0.1192                                      

[9]	train-error:0.068667	eval-error:0.1208                                      

pramas: {'gamma': 0.2, 'max_depth': 7, 'min_child_weight': 3.0}, logloss: 0.2927
[0]	train-error:0.107867	eval-error:0.1424                                      

[1]	train-error:0.0952	eval-error:0.1412                                        

[2]	train-error:0.081067	eval-error:0.1312                                      

[3]	train-error:0.073467	eval-error:0.1268                                      

[4]	train-error:0.0684	eval-error:0.124                                         

[5]	train-error:0.060933	eval-error:0.1224                                      

[6]	train-error:0.055467	eval-error:0.1212                                      

 90%|█████████ | 9/10 [00:02<00:00,  3.89trial/s, best loss: 0.2799878002092242]

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[7]	train-error:0.050533	eval-error:0.114                                       

[8]	train-error:0.045067	eval-error:0.1092                                      

[9]	train-error:0.0416	eval-error:0.11                                          

pramas: {'gamma': 0.1, 'max_depth': 8, 'min_child_weight': 1.0}, logloss: 0.2798
100%|██████████| 10/10 [00:02<00:00,  3.88trial/s, best loss: 0.2797985085770488]
best params: {'gamma': 0.1, 'max_depth': 8, 'min_child_weight': 1.0}, score: 0.2798
